In [2]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchResults
from langchain.document_loaders import WebBaseLoader
from pydantic import BaseModel, Field
import os

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class SearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class DuckDuckGoScrapeToolArgsSchema(BaseModel):
    urls: list[str] = Field(description="The query you will search for")


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to search the query from Wikipedia.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        wp = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wp.run(query)


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to search the query from DuckDuckGo.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        result = ddg.run(query)
        # 획득한 결과에서 url을 뽑아내서 리스트로 채우는 코드
        urls = [chunk.split("]")[0] for chunk in result.split("link: ")][1:]
        return urls


class DuckDuckGoScrapeTool(BaseTool):
    name = "DuckDuckGoScrapeTool"
    description = """
    Use this tool to get the URL when searching with DuckDuckGo.
    It takes a list of URL as an argument.
    """
    args_schema: Type[DuckDuckGoScrapeToolArgsSchema] = DuckDuckGoScrapeToolArgsSchema

    def _run(self, urls):
        loader = WebBaseLoader(urls)
        docs = loader.load()
        return docs


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        DuckDuckGoScrapeTool(),
    ],
)

prompt = "Research about the XZ backdoor"

result = agent.invoke(prompt)

base_dir = "./.cache/agent_results/"
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
with open(f"{base_dir}result.txt", "w") as f:
    f.write(f"Input: {result['input']}\n\nOutput:\n{result['output']}")



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously LZM